In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

2024-02-26 08:54:05.564949: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 08:54:05.567570: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 08:54:05.597086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 08:54:05.597143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 08:54:05.598166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

Ready to generate train and test datasets


In [4]:
model = joblib.load("/home/jovyan/MQTTset/dl/new_dl/dnn.joblib")

2024-02-26 08:54:27.050000: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://4bde75d174fd488193988563518cdafa: INVALID_ARGUMENT: ram://4bde75d174fd488193988563518cdafa is a directory.


In [5]:
x_test_trimmed = x_test[20000:24000]
y_test_trimmed = y_test[20000:24000]

In [6]:
np.unique(y_test_trimmed)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [12]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=6,
    input_shape=(33,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [28]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.01, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/3492240313.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [06:43<00:00,  9.92it/s]


In [29]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 675us/step
Accuracy: 0.03075
AUC: 0.5977531649983817

macro
Precision: 0.7226604084861837
Recall: 0.17413211909426238
F1 Score: 0.04824446909874045

weighted
Precision: 0.9432351180013594
Recall: 0.03075
F1 Score: 0.012614231873653984

Mean FNR: 0.8258678809057377
Mean TNR: 0.834193724169245
Mean FPR: 0.16580627583075505
Mean TPR: 0.17413211909426238


In [30]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.05, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/75040551.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [05:29<00:00, 12.14it/s]


In [31]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 631us/step
Accuracy: 0.3485
AUC: 0.6516032449413189

macro
Precision: 0.7474413459357026
Recall: 0.2413713302621756
F1 Score: 0.17777787335038572

weighted
Precision: 0.8396116964998585
Recall: 0.3485
F1 Score: 0.36923556202159247

Mean FNR: 0.7586286697378245
Mean TNR: 0.871488906695124
Mean FPR: 0.12851109330487598
Mean TPR: 0.2413713302621756


In [18]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/547066443.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:41<00:00, 39.28it/s]


In [19]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 601us/step
Accuracy: 0.21075
AUC: 0.6274331547806281

macro
Precision: 0.7711357135713571
Recall: 0.245080730664931
F1 Score: 0.15295355545358078

weighted
Precision: 0.8188666645664566
Recall: 0.21075
F1 Score: 0.22072009049771715

Mean FNR: 0.754919269335069
Mean TNR: 0.8526321393696007
Mean FPR: 0.1473678606303993
Mean TPR: 0.245080730664931


In [20]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.15, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/971322502.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:32<00:00, 43.20it/s]


In [21]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 566us/step
Accuracy: 0.0255
AUC: 0.6438166216535608

macro
Precision: 0.6801532823632271
Recall: 0.16812865497076024
F1 Score: 0.010770823592278406

weighted
Precision: 0.9484737778335844
Recall: 0.0255
F1 Score: 0.0016810221984726028

Mean FNR: 0.8318713450292398
Mean TNR: 0.8333736483363973
Mean FPR: 0.1666263516636027
Mean TPR: 0.16812865497076024


In [22]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.2, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/1386664024.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:36<00:00, 41.54it/s]


In [23]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 592us/step
Accuracy: 0.3805
AUC: 0.7126333807906843

macro
Precision: 0.7915835592984651
Recall: 0.2563469583255112
F1 Score: 0.17852734719076527

weighted
Precision: 0.8298673761211346
Recall: 0.3805
F1 Score: 0.4153876213787354

Mean FNR: 0.7436530416744888
Mean TNR: 0.8722178097783703
Mean FPR: 0.12778219022162976
Mean TPR: 0.2563469583255112


In [24]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.25, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/398718751.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:05<00:00, 61.03it/s]


In [25]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 755us/step
Accuracy: 0.22475
AUC: 0.6496994529705419

macro
Precision: 0.7582032530269119
Recall: 0.22535348007808495
F1 Score: 0.12408275310940663

weighted
Precision: 0.7881510298277968
Recall: 0.22475000000000003
F1 Score: 0.2109756706221486

Mean FNR: 0.774646519921915
Mean TNR: 0.8483584835145246
Mean FPR: 0.15164151648547539
Mean TPR: 0.22535348007808495


In [26]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_17946/2298617831.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:06<00:00, 60.15it/s]


In [27]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 685us/step
Accuracy: 0.03225
AUC: 0.5512324504536124

macro
Precision: 0.6520840971641712
Recall: 0.18246956274574824
F1 Score: 0.04696871063079477

weighted
Precision: 0.9409915696907035
Recall: 0.03225
F1 Score: 0.010976947860791534

Mean FNR: 0.8175304372542517
Mean TNR: 0.83429481107281
Mean FPR: 0.16570518892718997
Mean TPR: 0.18246956274574824
